In [1]:
import pandas as pd
import Levenshtein

In [2]:
words = pd.read_csv('words.csv')

In [3]:
words.head()

,Français,Hiragana,Kanji
0,enseignant,きょうし,教室
1,étudiant,がくせい,学生
2,employé d'une compagnie,かいしゃいん,会社員
3,employé (avec nom de compagnie),しゃいん,社員
4,employé de banque,ぎんこういん,銀行員


In [4]:
shuffled_words = words.sample(frac=1).reset_index(drop=True)

In [5]:
def is_similar_japanese_answer(user_answer, correct_answer):
    similarity_threshold = 0.8
    distance = Levenshtein.distance(user_answer.lower(), correct_answer.lower())
    similarity = 1 - (distance / max(len(user_answer), len(correct_answer)))
    return similarity >= similarity_threshold

In [6]:
def is_similar_french_answer(user_answer, correct_answer):
    similarity_threshold = 0.7
    if pd.isna(correct_answer):
        return False
    user_answer = user_answer.lower()
    correct_answer = correct_answer.lower()
    substring_match = user_answer in correct_answer or correct_answer in user_answer
    if substring_match:
        return True
    distance = Levenshtein.distance(user_answer, correct_answer)
    similarity = 1 - (distance / max(len(user_answer), len(correct_answer)))
    
    return similarity >= similarity_threshold

In [7]:
shuffled_sample = shuffled_words.sample(n=20)
shuffled_sample

,Français,Hiragana,Kanji
157,carte de visite,めいし,名称
108,produit,せいひん,製品
95,spécial express,とっきゅう,特急
67,restaurant / cantine,しょくどう,食堂
22,accueil,うけつけ,受付
31,passer [un coup de téléphone],かけます,NaN
140,saison,きせつ,季節
113,par bateau,ふなびん,船便
137,changer (de train),のりかえます,乗り換えます
115,adresse,じゅうしょ,住所


In [ ]:
seen_fr = []
correct_count = 0
score_fr = 0
shuffled_sample_jp = shuffled_sample.sample(frac=1).reset_index(drop=True)

for i in range(len(shuffled_sample_jp)):
    if shuffled_sample_jp['Français'].iloc[i] not in seen_fr:
        ask_h = input(f"Comment dire {shuffled_sample_jp['Français'].iloc[i]} en hiragana ?")
        seen_fr.append(shuffled_sample_jp['Français'].iloc[i])

        if is_similar_japanese_answer(ask_h, shuffled_sample_jp['Hiragana'].iloc[i]):
            kanji_value = shuffled_sample_jp['Kanji'].iloc[i]
            if pd.notna(kanji_value):
                ask_k = input("Et en kanji ?")
                if is_similar_japanese_answer(ask_k, kanji_value):
                    correct_count += 1
                    score_fr += 1
                    print('Correct !')
                    print(f'Bonnes réponses : {score_fr}/{len(shuffled_sample_jp)}')
                else:
                    print(f"Incorrect ! La bonne réponse est : {kanji_value}")
                    print(f'Bonnes réponses : {score_fr}/{len(shuffled_sample_jp)}')
            else:
                correct_count += 1
                score_fr += 1
                print('Correct !')
                print(f'Bonnes réponses : {score_fr}/{len(shuffled_sample_jp)}')
        else:
            print(f"Incorrect ! La bonne réponse est : {shuffled_sample_jp['Hiragana'].iloc[i]}")
            print(f'Bonnes réponses : {score_fr}/{len(shuffled_sample_jp)}')

if correct_count == len(shuffled_sample_jp):
    print("Félicitations ! Vous avez répondu correctement à toutes les réponses.")
else:
    print(f"Vous avez répondu correctement à {score_fr} des {len(shuffled_sample_jp)} questions.")

Comment dire enlever (chaussures/vêtement) en hiragana ?ぬぎます
Et en kanji ?脱ぎます
Correct !
Bonnes réponses : 1/20
Comment dire contemplation fleurs de cerisiers en hiragana ?おはなみ
Et en kanji ?お花見
Correct !
Bonnes réponses : 2/20


In [21]:
seen_k=[]
correct_count = 0
score_jp = 0
shuffled_sample_fr = shuffled_sample.sample(frac=1).reset_index(drop=True)
for i in range(len(shuffled_sample_fr)):
    kanji = shuffled_sample_fr['Kanji'].iloc[i]
    if pd.notna(kanji) and kanji not in seen_k:
        ask_h = input(f"Que veut dire ce kanji : {kanji}?")
        if is_similar_french_answer(ask_h, shuffled_sample_fr['Français'].iloc[i]):
            seen_k.append(kanji)
            correct_count += 1
            score_jp += 1
            print(f"Correct ! C'était bien {shuffled_sample_fr['Français'].iloc[i]}")
            print(f'Bonnes réponses : {score_jp}/{len(shuffled_sample_fr)}')
        else:
            print(f"Incorrecte ! La bonne réponse était : {shuffled_sample_fr['Français'].iloc[i]}")
            print(f'Bonnes réponses : {score_jp}/{len(shuffled_sample_fr)}')

if correct_count == len(shuffled_sample_fr):
    print("Félicitations ! Vous avez répondu correctement à toutes les réponses.")
else:
    print(f"Vous avez répondu correctement à {score_jp} des {len(shuffled_sample_fr)} questions.")

Que veut dire ce kanji : 留学生?étudiant étranger
Correct ! C'était bien étudiant étranger
Bonnes réponses : 1/20
Que veut dire ce kanji : お釣り?monnaie
Correct ! C'était bien monnaie
Bonnes réponses : 2/20
Que veut dire ce kanji : 封筒?enveloppe
Correct ! C'était bien enveloppe
Bonnes réponses : 3/20
Que veut dire ce kanji : 医者?médecin
Correct ! C'était bien médecin
Bonnes réponses : 4/20
Que veut dire ce kanji : 教えます?
Correct ! C'était bien enseigner / apprendre
Bonnes réponses : 5/20
Que veut dire ce kanji : 金額?montant
Correct ! C'était bien montant/somme d'argent
Bonnes réponses : 6/20
Que veut dire ce kanji : 借ります?emprunter
Correct ! C'était bien emprunter
Bonnes réponses : 7/20
Que veut dire ce kanji : 泊まります?descendre
Correct ! C'était bien descendre [à l'hôtel]
Bonnes réponses : 8/20
Que veut dire ce kanji : 庭?jardin
Correct ! C'était bien jardin
Bonnes réponses : 9/20
Que veut dire ce kanji : 部長?chef de département
Correct ! C'était bien chef de département
Bonnes réponses : 10/20
Que